Notebook to split data

# Imports

In [1]:
import os, os.path
import pandas as pd
from xml.dom import minidom
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Generate DF from annotations

In [2]:
# Generate empty dataframe

columns_names_df = ["task_name","img_number","usable_img","obj_detected_number","detected_ojects_number"]
df = pd.DataFrame(columns = columns_names_df)
df

,task_name,img_number,usable_img,obj_detected_number,detected_ojects_number


In [3]:
path = '/home/acarlier/OrnithoMate/p0133_bird_data/annotations/'
tasks_dir = sorted(os.listdir(path))
unwanted_list = ['unknown', 'human', 'noBird']

def get_info_from_xml(xml_path):
    mydoc = minidom.parse(xml_path)
    items = mydoc.getElementsByTagName('object')
    nb_obj = len(items)
    nb_obj_real = 0
    birds = []
    for k in range(nb_obj):
        bird = items[k].childNodes[7].childNodes[1].childNodes[3].childNodes[0].data
        if bird not in unwanted_list:
            birds.append(bird)
            nb_obj_real += 1
        if bird == '1L':
            print(xml_path)
            
    return nb_obj_real, birds


total_img = 0
total_obj = 0
total_obj_dict = {}
for i in range(len(tasks_dir)):
    # Pour chaque tâche
    task_name = tasks_dir[i]
    
    task_dict = {}
    
    complete_path = path + task_name + '/Annotations/bird/' + task_name + '/'
    # Lister toutes les images de la tâche
    image_paths = sorted(os.listdir(complete_path))

    nb_img_real = 0
    nb_obj_real = 0
    for img in range(len(image_paths)):
        nb_obj, birds = get_info_from_xml(complete_path + image_paths[img])
        
        if nb_obj > 0:
            nb_img_real += 1
            nb_obj_real += len(birds)
            
            for b in birds:
                if b in task_dict:
                    task_dict[b] += 1
                else:
                    task_dict[b] = 1
                    
                if b in total_obj_dict:
                    total_obj_dict[b] += 1
                else:
                    total_obj_dict[b] = 1
    
    total_img += nb_img_real
    total_obj += nb_obj_real        
    #print(task_name, len(image_paths), nb_img_real, nb_obj_real, task_dict)
    df = df.append({'task_name': task_name, 'img_number':len(image_paths),
                   'usable_img':nb_img_real,'obj_detected_number':nb_obj_real,'detected_ojects_number':task_dict},ignore_index=True)
#print('Total images ', total_img, total_obj, total_obj_dict)

/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041

/home/acarlier/OrnithoMate/p0133_bird_data/annotations/task_20210611_Lab/Annotations/bird/task_20210611_Lab/2021-06-11-13-46-55.xml
/home/acarlier/OrnithoMate/p0133_bird_data/annotations/task_20210611_Lab/Annotations/bird/task_20210611_Lab/2021-06-11-13-56-43.xml


/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041/2503037352.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'task_name': task_name, 'img_number':len(image_paths),
/tmp/ipykernel_67041

In [4]:
df.head()

,task_name,img_number,usable_img,obj_detected_number,detected_ojects_number
0,task_05-01-2021,948,740,812,"{'MESCHA': 287, 'SITTOR': 182, 'MESBLE': 69, '..."
1,task_06-01-2021,324,225,268,"{'MESBLE': 22, 'MESCHA': 135, 'SITTOR': 32, 'M..."
2,task_18-01-2021,297,244,245,"{'MESNON': 73, 'ROUGOR': 5, 'MESBLE': 91, 'MES..."
3,task_19-01-2021,737,543,554,"{'MESCHA': 344, 'MESNON': 79, 'MESBLE': 74, 'R..."
4,task_2021-03-01_07,162,124,130,"{'MESBLE': 6, 'MESCHA': 17, 'VEREUR': 107}"


# Add station name and split type to DF

In [5]:
df.shape

(47, 5)

In [6]:
df_task = pd.read_csv('Ornithotasks - CVAT_task.csv')
df_task = df_task.rename(columns={"Task name": "task_name"})
print(df_task.head(2))
df_task.shape

               task_name     Site         Day Day2  Split
0  task_20210623_balacet  balacet  23-06-2021  NaN  TRAIN
1  task_20210624_balacet  balacet  24-06-2021  NaN  TRAIN


(48, 5)

In [7]:
df_difference = df.merge(df_task, how = 'outer' ,indicator=False)
df_difference.head()

,task_name,img_number,usable_img,obj_detected_number,detected_ojects_number,Site,Day,Day2,Split
0,task_05-01-2021,948,740,812,"{'MESCHA': 287, 'SITTOR': 182, 'MESBLE': 69, '...",Gajan,05-01-2021,NaN,TRAIN
1,task_06-01-2021,324,225,268,"{'MESBLE': 22, 'MESCHA': 135, 'SITTOR': 32, 'M...",Gajan,06-01-2021,NaN,TRAIN
2,task_18-01-2021,297,244,245,"{'MESNON': 73, 'ROUGOR': 5, 'MESBLE': 91, 'MES...",Gajan,18-01-2021,NaN,TRAIN
3,task_19-01-2021,737,543,554,"{'MESCHA': 344, 'MESNON': 79, 'MESBLE': 74, 'R...",Gajan,09-01-2021,NaN,TRAIN
4,task_2021-03-01_07,162,124,130,"{'MESBLE': 6, 'MESCHA': 17, 'VEREUR': 107}",Francon,01-03-2021,NaN,TRAIN


# Get percentages split values

In [8]:
def extract_from_split_value(df, column_value_selected, column_split_value, split_value):
    df_split_value = df.loc[df[column_split_value] == split_value]
    values_selected = df_split_value[column_value_selected]
    return values_selected

def get_percentage_split_value(df, column_value_selected, column_split_value):
    test = extract_from_split_value(df_difference, 'obj_detected_number', 'Split', 'TEST')
    train = extract_from_split_value(df_difference, 'obj_detected_number', 'Split', 'TRAIN')
    count_test = 0
    count_train = 0
    
    for value in test:
        if isinstance(value, int):
            count_test += value
    for value in train:
        if isinstance(value, int):
            count_train += value
    print(f"Train: {count_train} Test: {count_test}")
    total_value_selected = count_train + count_test
    percentage_train = round(count_train/total_value_selected,2)
    percentage_test = round(count_test/total_value_selected,2)
    return percentage_train,percentage_test

get_percentage_split_value(df_difference, 'obj_detected_number', 'Split')

Train: 11171 Test: 2917


(0.79, 0.21)

# Make validation dataset from DF train

In [9]:
def random_split_train_in_validation(df_input, column_split):
    df = df_input.copy()
    df_train = df.loc[df[column_split] == 'TRAIN']
    df_train_lenght = df_train.shape[0]
    df_validation = df_train.sample(round(df_train_lenght/10))
    df_validation['Split'] = df_validation['Split'].replace('TRAIN','VALIDATION')
    print('Indexes of validation tasks', list(df_validation.index))
    df.loc[df_validation.index] = df.loc[df_validation.index].replace('TRAIN','VALIDATION')
    return df
df_split = random_split_train_in_validation(df_difference,'Split')

Indexes of validation tasks [13, 16, 24, 22]


In [10]:
df_split

,task_name,img_number,usable_img,obj_detected_number,detected_ojects_number,Site,Day,Day2,Split
0,task_05-01-2021,948,740,812,"{'MESCHA': 287, 'SITTOR': 182, 'MESBLE': 69, '...",Gajan,05-01-2021,NaN,TRAIN
1,task_06-01-2021,324,225,268,"{'MESBLE': 22, 'MESCHA': 135, 'SITTOR': 32, 'M...",Gajan,06-01-2021,NaN,TRAIN
2,task_18-01-2021,297,244,245,"{'MESNON': 73, 'ROUGOR': 5, 'MESBLE': 91, 'MES...",Gajan,18-01-2021,NaN,TRAIN
3,task_19-01-2021,737,543,554,"{'MESCHA': 344, 'MESNON': 79, 'MESBLE': 74, 'R...",Gajan,09-01-2021,NaN,TRAIN
4,task_2021-03-01_07,162,124,130,"{'MESBLE': 6, 'MESCHA': 17, 'VEREUR': 107}",Francon,01-03-2021,NaN,TRAIN
5,task_2021-03-01_08,317,108,115,"{'MESCHA': 61, 'MESBLE': 9, 'VEREUR': 45}",Francon,01-03-2021,NaN,TRAIN
6,task_2021-03-01_09,444,196,209,"{'VEREUR': 68, 'MESCHA': 68, 'PINARB': 6, 'MOI...",Francon,01-03-2021,NaN,TRAIN
7,task_2021-03-01_10,525,169,174,"{'PINARB': 20, 'MESCHA': 45, 'VEREUR': 93, 'ME...",Francon,01-03-2021,NaN,TRAIN
8,task_2021-03-01_11,278,128,130,"{'MESCHA': 103, 'MESBLE': 17, 'PINARB': 5, 'VE...",Francon,01-03-2021,NaN,TRAIN
9,task_2021-03-01_12,158,115,117,"{'MESCHA': 78, 'VEREUR': 33, 'MESBLE': 5, 'PIN...",Francon,01-03-2021,NaN,TRAIN


# Get numbers objects depending on split value
Ex: MESCHA: 600 Train; 200 Validatio; 40 Test

In [18]:
def check_data_type(data):
    if type(data) is str:
        return data
    elif type(data) is dict:
        return data

def get_number_object_depending_on_split_value(df):
    dict_birds = {}
    df_clean = df[['detected_ojects_number', 'Split']]
    for col_name, data in df_clean.items():
        #data = value[0]
        print(type(data))
        print(data[0])
        
        
        
        """
        if type(data) is str:
            split_value_type = data
            print("Val type",split_value_type)
        if type(data) is dict:
            print(data)
            species = []
            numbers = []
            for specie, number in data.items():
                species.append(specie)
                numbers.append(number)
            print(species,numbers)
        
        #dict_birds.fromkeys(species, None)
        #specie_dict = {specie:{split_value_type:number}}
        #print(dict_birds)
        """  

get_number_object_depending_on_split_value(df_split)

<class 'pandas.core.series.Series'>
{'MESCHA': 287, 'SITTOR': 182, 'MESBLE': 69, 'MESNON': 100, 'PINARB': 2, 'ACCMOU': 147, 'ROUGOR': 25}
<class 'pandas.core.series.Series'>
TRAIN
